In [1]:
from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
import uuid
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults

In [2]:
load_dotenv(override=True)

True

In [21]:
tool = TavilySearchResults(max_results=5)

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


In [22]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature = 0.2
)

llm_with_tool = llm.bind_tools([tool])

In [23]:
def chatnode(state: State):
    return {"messages": [llm_with_tool.invoke(state["messages"])]}

 `tools_condition`

---

### **What this function does**

`tools_condition` is a **decision-making helper** in LangGraph-based agents that follow the **ReAct** (Reason + Act) pattern.

It:

1. Looks at the **latest AI message** in your current conversation state.
2. Checks: **Does this AI message contain a tool call?**

   * If **yes** → it tells the workflow to go to the `"tools"` node (execute tools).
   * If **no** → it tells the workflow to go to `"__end__"` (stop).

This is how your agent decides **“Should I run a tool or finish the conversation?”**

---

### **Why it’s useful**

* In LangGraph, workflows often have **conditional edges** that route execution.
* You don’t want to manually write this “check latest message for tool calls” logic every time.
* This function **centralizes** that check and works for **multiple state formats**:

  * **List of messages** (`[msg1, msg2, ...]`)
  * **Dictionary** with a `messages` key (`{"messages": [...]}`)
  * **Pydantic model** with a `.messages` attribute

---



In [24]:
graph_builder = StateGraph(State)

graph_builder.add_node("Chatnode", chatnode)
graph_builder.add_node("tools", ToolNode([tool]))

graph_builder.add_edge(START, "Chatnode")
graph_builder.add_conditional_edges("Chatnode",
                                    tools_condition,
                                    {"tools": "tools",
                                     END:END})
graph_builder.add_edge("tools","Chatnode")

memory = MemorySaver()

graph = graph_builder.compile(checkpointer = memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
print("Welcome to the Chatbot. If you want to exit then type 'exit'.")
while True:

    user_input = input("Human: ")
    if user_input =="exit":
        print("see yaa.")
        break
    response = graph.invoke({"messages": [{
                                            "role": "user",
                                           "content":user_input}]},
                                           config=config)
    print("AI:",response["messages"][-1].content)
    print("__________________________________________________________________")

Welcome to the Chatbot. If you want to exit then type 'exit'.
AI: As of 2023, the Gross Domestic Product (GDP) of India is approximately **$3.568 trillion**. This represents a growth of about **6.38%** from the previous year, 2022, when the GDP was around **$3.353 trillion**. 

Additionally, the GDP per capita for India in 2023 is estimated to be **$2,480.79**.

For more detailed information, you can refer to the following sources:
- [Macrotrends - India GDP](https://www.macrotrends.net/global-metrics/countries/ind/india/gdp-gross-domestic-product)
- [Statista - GDP of India](https://www.statista.com/statistics/263771/gross-domestic-product-gdp-in-india/)
AI: In 2025, India's GDP is projected to be approximately **$4.187 trillion**. This reflects a continued growth trend, with the economy expected to grow at a rate of around **6.5%** for the fiscal year 2024-2025.

For more detailed insights, you can refer to the following sources:
- [Trading Economics - India GDP Growth](https://tradi